# 笔记本2：物理仿真与机器人加载

**学习目标：**
1. 使用 `Parser` 和预设的机器人文件（URDF、SDF、MJCF）将真实机器人加载到 Drake
2. 理解 Drake 的物理仿真引擎（`MultibodyPlant`）和几何引擎（`SceneGraph`）
3. 创建自定义三维资产（你的姓名首字母！）并学习 `URDF` 和 `SDF` 的编写
4. 使用 Meshcat 可视化机器人、物体和仿真
5. 在 Drake 中构建包含自定义物体的完整机器人仿真

**你将实现的内容：** 一个完整的仿真场景，包含 IIWA14 机械臂与自定义姓名首字母资产的交互！

---


## 环境配置与导入

首先导入用于物理仿真、机器人加载和可视化的 Drake 功能。


In [ ]:
from pathlib import Path

import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    BasicVector,
    Context,
    Diagram,
    DiagramBuilder,
    LeafSystem,
    MeshcatVisualizer,
    ModelInstanceIndex,
    MultibodyPlant,
    Parser,
    RigidTransform,
    Simulator,
    StartMeshcat,
)
from pydrake.visualization import ModelVisualizer

from manipulation import running_as_notebook
from manipulation.exercises.grader import Grader
from manipulation.exercises.intro.test_physics_simulation_and_robots import (
    TestPhysicsSimulationDiagramStructure,
    TestPhysicsSimulationFullSystem,
    TestPhysicsSimulationSimpleController,
    TestPhysicsSimulationVerification,
)
from manipulation.letter_generation import create_sdf_asset_from_letter
from manipulation.utils import RenderDiagram

## Meshcat 可视化

在开始之前，让我们配置 Meshcat，可视化工具。

**Meshcat** 是 Drake 的主力三维可视化工具，在你的网页浏览器中运行。它可以实时交互式地可视化机器人、物体和仿真环境。Meshcat 让你可以：

- **可视化三维机器人和场景**，带有真实几何和材质
- **实时查看仿真过程**
- **交互操作三维场景**（旋转、缩放、平移）
- **录制视频**，用于作业提交
- **调试仿真**，直观了解实际发生了什么

你会经常用 meshcat 来可视化和调试你的代码！


In [ ]:
# Start meshcat for visualization
meshcat = StartMeshcat()

**点击上方链接，在浏览器中打开 Meshcat！本笔记本将一直使用同一个 Meshcat 窗口，请保持其打开！**

---

## 第一部分：加载 IIWA14 机器人

首先，让我们加载一个真实机器人！**IIWA14** 是 KUKA 公司出品的 7 自由度工业机械臂。Drake 内置了许多可直接加载的机器人模型，本部分我们就会用到。你可以在[这里](https://github.com/RobotLocomotion/models)找到所有可用的 Drake 模型列表。

**你的任务：** 使用 Drake 的物理引擎加载并可视化 IIWA14 机器人。

**核心概念：**
- `Parser`：将机器人或资产的描述文件（如 URDF/SDF/MJCF）加载到物理引擎中
- `MultibodyPlant`：Drake 的主要机器人和物体物理仿真引擎，是我们所有仿真的核心
- `SceneGraph`：Drake 用于管理几何体注册、几何查询、碰撞检测等的模块
- **基座焊接**：将机器人基座固定到世界坐标系很重要，否则它会掉下去！

**参考：** 本部分紧跟教材 [第二章](https://manipulation.csail.mit.edu/robot.html) 的 [Drake Simulation 示例](https://deepnote.com/workspace/Drake-0b3b2c53-a7ad-441b-80f8-bf8350752305/project/Tutorials-2b4fc509-aef2-417d-a40d-6071dfed9199/notebook/simulation-1ba6290623e34dbbb9d822a2180187c1) —— 重点参考 **“Simulating the (passive) iiwa”** 和 **“Visualizing the scene”** 两节。建议在开始实现前先浏览一下这些内容！

In [ ]:
# TODO：加载并可视化 IIWA14 机器人


def create_IIWA14_diagram() -> tuple[Diagram, MultibodyPlant, ModelInstanceIndex]:
    # 提示：参见上方 Drake Simulation 示例，“Visualizing the scene”小节
    # TODO：创建 DiagramBuilder

    # TODO：添加 MultibodyPlant 和 SceneGraph，时间步长为 1e-4
    # 提示：有一个辅助函数可以帮你自动创建并连接这两个系统。

    # TODO：创建 Parser 用于加载机器人模型
    # 注意：在上方 Drake Simulation 示例中，parser 用完即丢弃，
    #       这里我们会保存 parser 以便后续添加多个模型。

    # TODO：用 parser 从 Drake 模型库加载 IIWA14 机器人模型。
    # 提示：教程中的 IIWA14 模型没有碰撞几何体，这里我们加载带有碰撞几何体的模型：
    #       "package://drake_models/iiwa_description/urdf/iiwa14_primitive_collision.urdf"
    # 提示：parser 的 `AddModelsFromUrl` 方法返回一个模型引用列表（本例长度为 1）。
    #       需要将第一个元素保存为变量 `iiwa`。
    #       （严格来说，引用类型为 `ModelInstanceIndex`，不用担心细节。）

    # TODO：将机器人基座（"iiwa_link_0"）焊接到世界坐标系

    # TODO：将 MeshcatVisualizer 添加到 builder 并连接到 SceneGraph

    # TODO：finalize plant（仿真前必须）

    # TODO：构建完整 diagram

    # TODO：返回 diagram、plant 和 iiwa 模型实例
    return None, None, None  # 实现后请移除此行


diagram, plant, iiwa = create_IIWA14_diagram()

在上面的代码中，我们加载了与教程不同的模型，因为我们需要带有碰撞几何体的模型。实际上，Drake 有许多可直接使用的模型，全部可在 [Github 仓库](https://github.com/RobotLocomotion/models) 找到，Parser 的 url 格式为 `package://drake_models/{models 仓库中的文件路径}`。

用下方代码测试你的实现：

In [ ]:
Grader.grade_output([TestPhysicsSimulationDiagramStructure], [locals()], "results.json")
Grader.print_test_results("results.json")

**我们还可以可视化模块图，方便你查看整体结构！**

- 注意 `MultibodyPlant`、`SceneGraph` 和 `MeshcatVisualizer` 都是模块图中的系统。
- 请务必观察各系统端口之间的连接关系！

In [ ]:
RenderDiagram(diagram, max_depth=1)

我们还可以为 diagram 创建一个默认 context 并打印出来，帮助我们了解当前系统状态！

**提醒：** context 保存了“全部状态”，即 diagram 内所有系统的动力学信息、仿真时间等。

In [ ]:
# HINT: You already learned how to do this in the first notebook!
# TODO: Create a context for the diagram

# TODO: Print the context

哇！现在 context 里信息量很大。我们只打印 plant 的 context 看看。

（**提示：** 对于给定的 `system` 和 diagram context，可以用 `system.GetMyContextFromRoot(diagram_context)` 获取该系统的 context）

- plant 有多少个（普通）状态？和你对 IIWA14 自由度的预期一致吗？

In [ ]:
# TODO: Get the context of the plant from the diagram_context
# HINT: This is important!
#       Use the method plant.GetMyContextFromRoot(diagram_context),
#       similar to what you did for the pendulum in the first notebook!

# TODO: Print the plant context

---

## 第二部分：仅仿真 IIWA14 

很好，IIWA14 已经加载到 Drake！

现在我们设置初始条件并仿真该系统（目前只有 IIWA）。仿真时请关注 meshcat 可视化窗口！

**参考：** 本部分继续参考 [Drake Simulation 示例](https://deepnote.com/workspace/Drake-0b3b2c53-a7ad-441b-80f8-bf8350752305/project/Tutorials-2b4fc509-aef2-417d-a40d-6071dfed9199/notebook/simulation-1ba6290623e34dbbb9d822a2180187c1)。

In [ ]:
# TODO：用你实现的 create_IIWA14_diagram 函数进行机器人仿真


def simulate_IIWA14(
    q0: np.ndarray, simulation_time: float = 3.0, set_target_realtime_rate: bool = True
) -> np.ndarray:
    # 提示：你在第一个笔记本已经学会了大部分内容！

    # TODO：用 create_IIWA14_diagram 创建仿真系统

    # TODO：创建 diagram context

    # 提示：参见上方 Simulation 示例，“Visualizing the scene”小节
    # TODO：用 diagram context 获取 plant 的 context
    # 提示：很重要！plant context 是 diagram context 的子集，
    #       只需修改 plant context 设置初始关节位置。
    # 提示：之前用的是 `GetMyContextFromRoot`，现在要修改 context，
    #       应用 `GetMyMutableContextFromRoot`（返回可变 context）。

    # TODO：将机器人关节初始位置设置为 q0（弧度）

    # TODO：将执行器输入设为零（被动仿真）
    # 提示：参见上方 Drake Simulation 示例，方法一致。
    # 提示：IIWA14 有 7 个关节，需要传入 7 个力矩值的向量

    # TODO：创建仿真器

    # TODO：设置可视化实时速率（便于观察）
    # if set_target_realtime_rate:
    # 提示：用 simulator.set_target_realtime_rate(1.0)

    # TODO：运行仿真

    # TODO：返回最终关节位置
    # 提示：之前用 logger 记录系统输出历史，这里只需用
    #       `plant.GetPositions(plant_context)`（仿真结束后）获取最终关节位置。

    return np.zeros(7)

Okay! If everything is working as expected, we should now be able to simulate our system. Let us try by running the code below (if everything is working correctly, you should see the IIWA14 swinging passively in your meshcat window).


In [ ]:
# Test your implementation
q_initial = np.array([0, 0, 0, 0, 0, 0, 0])

print("Starting robot simulation...")
print(f"Check your Meshcat window ({meshcat.web_url()}) to see the robot simulation!")
print("     - If the robot is not moving, the simulation already finished.")
print(
    "     - Try running this cell again (or increase the simulation time), with the meshcat window open!"
)
q_final = simulate_IIWA14(
    q_initial, simulation_time=20.0 if running_as_notebook else 0.1
)

print("Robot simulation completed!")
print(f"   Initial joint positions: {q_initial}")
print(f"   Final joint positions: {q_final}")

好的，成功了。不过 IIWA 只是被动摆动。让我们写一个简单的控制器，至少能让 IIWA 基本保持在期望位置！我们会像第一个笔记本那样，把（非常简单的）控制器实现为 `LeafSystem`。注意 Drake 已经为你实现了许多高级控制器，实际项目中你通常会用那些或者写更好的控制器。这里我们只是为了学习如何将自定义 `LeafSystem` 控制器与 `MultibodyPlant` 连接，写一个非常简单（甚至有点“蠢”）的控制器。

In [ ]:
class SimpleController(LeafSystem):
    def __init__(self, gain: float, q_desired: np.ndarray) -> None:
        # TODO：定义一个 LeafSystem，输入为 14 维（IIWA 状态，即 7 个位置 + 7 个速度），
        #       输出为 7 维（IIWA 关节力矩）。输出由 `ComputeTorque` 方法计算。
        # 提示：第一个笔记本你已经会了！
        pass

    def ComputeTorque(self, context: Context, output: BasicVector) -> None:
        # TODO：实现一个简单控制器，输出力矩 `tau = -gain * (q - q_desired)`。
        # 提示：需要从状态中提取前 7 个元素作为关节位置 `q`。
        # 提示：剩下的你已经会了！
        pass

Run the code below to test your implementation:

In [ ]:
Grader.grade_output([TestPhysicsSimulationSimpleController], [locals()], "results.json")
Grader.print_test_results("results.json")

In [ ]:
# TODO：加载并可视化 IIWA14 机器人并连接你的控制器


def create_IIWA14_diagram_with_controller(
    controller_gain: float, q_desired: np.ndarray
) -> tuple[Diagram, MultibodyPlant, ModelInstanceIndex]:
    # TODO：构建包含 IIWA 和控制器的 diagram。
    #       结构与 create_IIWA14_diagram 基本一致，只是多了控制器连接。

    # TODO：创建 diagram builder，添加 multibody plant 和 scene graph，
    #       创建 parser，加载 IIWA，焊接基座。

    # TODO：plant.Finalize() 必须在连接控制器前调用！

    # TODO：用参数创建控制器

    # TODO：将 plant 的 IIWA 状态输出端口连接到控制器输入端口
    # 提示：用 `plant.get_state_output_port(iiwa)` 获取 IIWA 状态端口，
    #       用 `controller.{你的输入端口名}` 获取控制器输入端口。
    #       其中 `iiwa` 是 parser 返回的 IIWA 模型引用。

    # TODO：将控制器输出端口连接到 plant 的 IIWA 执行器输入端口
    # 提示：用 `controller.{你的输出端口名}` 获取控制器输出端口，
    #       用 `plant.get_actuation_input_port(iiwa)` 获取 plant 的执行器输入端口。

    # TODO：连接可视化器，构建 diagram。
    return None, None, None

Let us print the diagram to make sure that the connections are as expected. Make sure the the output port `iiwa14_state` from the `MultibodyPlant` system is connect to the input port of the controller, and that the output port of the controller is connected to `iiwa14_actuation` in `MultibodyPlant`!

In [ ]:
RenderDiagram(
    create_IIWA14_diagram_with_controller(10.0, np.array([1, 1, 0, 0, 0, 0, 0]))[0],
    max_depth=1,
)

Almost there! Now, just implement the `simulate_IIWA14` function again, but this time use your newly defined function `create_IIWA14_diagram_with_controller`:

In [ ]:
# TODO：用你实现的 create_IIWA14_diagram_with_controller 函数进行机器人仿真


def simulate_IIWA14_with_controller(
    q0: np.ndarray,
    controller_gain: float,
    q_desired: np.ndarray,
    simulation_time=3.0,
    set_target_realtime_rate=True,
) -> np.ndarray:

    return np.zeros(7)

Amazing! Now let us run a simulation. How high do you have to set the gain to make the IIWA keep its starting configuration?

In [ ]:
# Test your implementation
q_initial = np.array([0, 1.0, 0.3, 0.7, 0, 0, 0])

print("Starting robot simulation...")
print(f"Check your Meshcat window ({meshcat.web_url()}) to see the robot simulation!")
print("     - If the robot is not moving, the simulation already finished.")
print(
    "     - Try running this cell again (or increase the simulation time), with the meshcat window open!"
)
q_final = simulate_IIWA14_with_controller(
    q_initial,
    controller_gain=10,
    q_desired=q_initial,
    simulation_time=15 if running_as_notebook else 0.1,
    set_target_realtime_rate=running_as_notebook,
)

print("✅ Robot simulation completed!")
print(f"   Initial joint positions: {q_initial}")
print(f"   Final joint positions: {q_final}")

**NOTE:** You will notice that this controller does not work very well. That is because we are only using a simple proportional controller (a "P-controller"), which only inputs a multiple of the tracking error. The purpose of this exercise is to teach you how to use Drake, but we will soon learn about much better control methods!

### VERIFICATION IN GRADESCOPE

Simulate your system with the following values, and copy/paste the final joint positions of the IIWA14:
- initial positions: $[0.2, 0.2, 0.2, 0, 0, 0, 0]$
- q_desired: $[0, 0, 0, 0, 0, 0, 0]$
- Controller gain: $120.0$
- Simulation time: $10 s$



In [ ]:
# TODO：用指定初始条件和仿真时间仿真 IIWA14

---

## 第三部分：用 SDFormat 创建自定义模型

让我们学习如何为仿真创建自定义物体！前两部分我们用的是预定义的 SDFormat（.sdf）文件加载 IIWA14。现在我们将用同样的格式从零创建一个简单的桌子，然后用提供的 API 生成更复杂的资产（你的姓名首字母），并演示如何将它们加载到仿真中。

**你的任务：** 
1. 从零创建一个简单桌子的 SDFormat 模型
2. 用提供的 API 为你的姓名首字母生成 URDF 文件

**核心概念：**
- `SDF`、`URDF`、`MJCF`：三种常用（且非常相似）的 XML 格式，用于描述仿真中的机器人和物体
- **可视几何体**：可视化时看到的部分（可以很精细）
- **碰撞几何体**：物理引擎用于接触计算的部分（应尽量简单）
- **惯性属性**：质量和惯性，实现真实物理行为

**参考：** 本部分参考 [Authoring a Multibody Simulation Tutorial](https://github.com/RobotLocomotion/drake/blob/master/tutorials/authoring_multibody_simulation.ipynb) —— 重点关注 **“Creating custom models”** 小节，教程演示了如何用 SDFormat 字符串创建圆柱体模型。建议先浏览该节内容再开始实现！


#### 步骤1：从零编写一个简单“桌子”的 SDF 文件

SDF、URDF 和 MJCF 文件其实很简单。这里你将从零编写一个简单“桌子”的 SDF 文件，帮助你今后自定义资产。

下方代码中我们直接在代码里定义 SDF 字符串，实际项目中通常会保存为单独的 `.sdf` 文件后加载到 Drake。

**警告：** SDFormat 对缩进很敏感，请参考教程示例的缩进！也可参考 [官方 SDFormat 文档](http://sdformat.org/tutorials)


In [ ]:
# TODO：为“桌子”创建一个简单的 SDFormat 字符串（一个扁平长方体即可）。
# 提示：紧跟教程中的圆柱体示例！
# 注意：后续我们会把它当作桌子放置姓名首字母资产，
#       所以要足够大！
# 提示：推荐尺寸 2m x 2m x 0.1m
# 提示：质量可设为 1kg，惯性张量可在网上查找长方体的公式，
#       不必完全精确，只要物理合理即可。
table_sdf = """<?xml version="1.0"?>
"""

很好！如果一切顺利，我们现在应该能可视化自定义资产了。我们用 Drake 内置的 `ModelVisualizer` 可视化（见教程“Creating Custom Models”小节）。写好下方代码后，你应该能在 Meshcat 窗口看到你的桌子！

**注意：** 观看完毕后请点击 MeshCat 中“Open Controls -> Stop Running”，这样虽然可视化停止但桌子仍可见。


In [ ]:
print(f"Meshcat URL: {meshcat.web_url()}")
# HINT: Follow the authoring tutorial's example for visualizing the cylinder!
# TODO: Create a ModelVisualizer instance

# TODO: Load your table_sdf string into the visualizer

# TODO: Run the visualizer to see your box in Meshcat
# NOTE: The tutorial uses `Run(loop_once=False)`, but you can just omit the argument.

# Make sure to click "Open Controls -> Stop Running" in MeshCat when you're done viewing

#### 步骤2：用你的姓名首字母创建自定义资产

本步骤你将用姓名首字母创建自定义资产。我们已为你提供了一个库，可以为给定字母生成 `.sdf` 文件，其结构和你上面写的桌子 SDF 很类似！

下方代码已为你准备好（生成字母文件可能需要约 30 秒）。你可以自由修改字体、尺寸和挤出深度！


In [ ]:
# Let us save our files to a new folder called `assets` in the `Files`
# section on the left
output_dir = Path("assets/")

# TODO: Insert your initials here!
your_initials = "BPG"

for letter in your_initials:
    create_sdf_asset_from_letter(
        text=letter,
        font_name="DejaVu Sans",
        letter_height_meters=0.2,
        extrusion_depth_meters=0.07,
        output_dir=output_dir / f"{letter}_model",
    )

很好。继续前请在左侧“文件”区查看 `assets/` 文件夹下生成的文件，结构如下：

```
{letter}_model/
├── {letter}_parts/
│   ├── convex_piece_000.obj
│   ├── convex_piece_001.obj
│   ├── convex_piece_002.obj
│   └── ...（更多凸包网格文件）
├── {letter}.obj
└── {letter}.sdf
```

例如字母 `A`：

```
A_model/
├── A_parts/
│   ├── convex_piece_000.obj
│   ├── convex_piece_001.obj
│   ├── convex_piece_002.obj
│   └── ...（更多凸包网格文件）
├── A.obj
└── A.sdf
```

`.obj` 文件描述字母整体三维网格，`convex_piece_XXX.obj` 文件描述字母的凸包部分：这些都是文本文件，包含顶点和面列表。可以尝试用 Deepnote 的“Open in raw mode”按钮打开看看。

`.sdf` 文件描述每个字母的资产，链接整体 `.obj` 文件作为可视几何体，凸包 `.obj` 文件作为碰撞几何体（每个 mesh 必须是凸的，所以每个字母会生成多个部分！）。


很好！字母已生成。现在用 Drake 的 `ModelVisualizer` 可视化它们，检查生成效果。补全下方代码，确保每个字母都能正确显示。

**注意**：点击 `Scene > drake` 可开启惯性和碰撞几何体的可视化，这对调试资产物理属性很有帮助！

In [ ]:
print(f"Meshcat URL: {meshcat.web_url()}")

# TODO: Change this to visualize the individual letters in your initials
letter_to_visualize = "P"  # One at a time!

# This is the path to the SDF file for the letter
letter_path = f"assets/{letter_to_visualize}_model/{letter_to_visualize}.sdf"

# TODO: Define a ModelVisualizer instance
# HINT: You know how to do this by now!

# TODO: Load the letter into the visualizer using the parser
# HINT: You can just pass the path to the letter file to the parser

# TODO: Run the visualizer

---

## 第四部分：完整仿真——机器人 + 你的自定义资产

**最终大作业！** 现在我们将把所学全部结合，创建一个包含 IIWA14 机器人和你自定义姓名首字母资产的完整仿真！

**你的任务：** 构建包含以下内容的物理仿真：
1. **IIWA14 机器人**：如前焊接到世界坐标系。
2. **你的自定义桌子**：焊接到世界坐标系，靠近机器人（但不重叠！）
3. **你的姓名首字母**：放在桌子上方。

你已经掌握了全部实现所需的知识！参考前面各部分的模式完成本节。完成后请录屏（.mp4 文件，大小不超过 500 MB）并上传到 Gradescope。

**最终目标：** 用录屏工具录制仿真场景，视频应展示你的姓名首字母落到桌子上，桌子焊接在机器人附近（但不重叠）。

**提示**：`WeldFrames` 的第三个参数是 `RigidTransform`（后续课程会详细讲）。现在你可以直接用 `RigidTransform([x,y,z])` 指定偏移量。若不确定函数签名请查阅[官方文档](https://drake.mit.edu/pydrake/pydrake.multibody.plant.html)。


In [ ]:
# Before you start, make sure to run the following cell to clear the meshcat window!
meshcat.Delete()
meshcat.DeleteAddedControls()

In [ ]:
# TODO：在这里实现你的完整仿真！


def simulate_full_system(
    initial_iiwa_positions: np.ndarray,
    initial_letter_poses: list[RigidTransform],
    table_pose: RigidTransform,
    simulation_time: float = 15.0,
) -> None:
    # TODO：创建 diagram builder，添加 multibody plant 和 scene graph

    # TODO：加载 IIWA14 机器人并焊接基座到世界坐标系

    # TODO：加载桌子并焊接到世界坐标系。
    #       别忘了用 `table_pose` 作为 `WeldFrames` 的第三个参数！

    # TODO：加载你的姓名首字母资产
    # 提示：用 'assets/...' 路径访问字母资产，方法同上！

    # TODO：添加可视化器

    # TODO：finalize plant 并构建 diagram

    # TODO：创建 diagram context 和 plant context

    # TODO：设置 IIWA14 关节初始位置。
    # 提示：现在场景中有多个自由物体，需指定
    #       设置的是 IIWA14 关节位置：
    # plant.SetPositions(plant_context, iiwa, initial_iiwa_positions)

    # TODO：加载字母并设置其初始位姿。
    # 提示：设置字母位姿需要新方法。
    #       因为字母是自由体，需要设置其位姿（位置+朝向）。
    #
    #       首先获取字母对应的 RigidBody，然后用 SetFreeBodyPose 方法设置位姿，
    #       参数为 RigidTransform。
    #
    #       例如字母 B：
    #       （注意：先加载字母，再 finalize plant，再设置位姿）
    #
    #               ```
    #               B_letter = parser.AddModels(f"assets/B_model/B.sdf")[0]
    #               ...
    #               plant.Finalize()
    #               ...
    #               body = plant.GetRigidBodyByName("B_body_link", B_letter)
    #               plant.SetFreeBodyPose(plant_context, body, initial_letter_poses[0])
    #               ```
    #
    #       （注意 SDF 中 body 名为 "B_body_link"，由字母生成函数硬编码。）

    #       按此模式为所有字母设置位姿！

    # TODO：将 plant 执行器输入固定为零

    # TODO：创建仿真器并运行 simulation_time 秒
    pass

Run the code below to visualize your simulation!

**Tip:** If you ever want to download a recording of your simulation as a `.html` file, append `/download` to the meshcat URL in your browser. Go ahead and try it for your simulation below!

In [ ]:
print(f"Meshcat URL: {meshcat.web_url()}")

# Notice that we have wrapped your function in a `meshcat.StartRecording()` and `meshcat.StopRecording()` block,
# followed by a `meshcat.PublishRecording()` call. This is a convenient way to make it possible to replay the
# simulation in meshcat after the simulation ends!
meshcat.StartRecording()
simulate_full_system(
    initial_iiwa_positions=np.array([-1.57, 0.1, 0, -1.2, 0, 1.6, 0]),
    initial_letter_poses=[
        # You can add rotations to the RigidTransform if you want, but we will learn more about that later!
        # The letters should be placed such that they are not in collision with each other, and such that
        # they fall onto the table.
        RigidTransform([0.7, 0.0, 1.0]),
        RigidTransform([0.9, 0.0, 1.0]),
        RigidTransform([1.1, 0.0, 1.0]),
    ],
    table_pose=RigidTransform(
        [0.5, 0.0, -0.05]
    ),  # this is a reasonable position for a 2m x 2m x 0.05m table, but feel free to change it!
    simulation_time=5.0 if running_as_notebook else 0.1,
)
meshcat.StopRecording()
meshcat.PublishRecording()

# GRADESCOPE 验证
恭喜你！本笔记本全部完成。现在在浏览器中回放仿真并将录屏（mp4，500MB 以下）上传到 Gradescope。

---

# 恭喜你！

你已成功完成 **笔记本2：物理仿真与机器人加载**！

### 总结

本笔记本你学会了：

- 用 Meshcat 可视化机器人、物体和仿真
- 用 `Parser` 和机器人描述文件加载真实机器人（本次只用 SDF，URDF 和 MJCF 也很类似！）
- 理解 Drake 的物理仿真引擎（`MultibodyPlant`）和几何引擎（`SceneGraph`）
- 创建自定义三维资产，学习 URDF/SDF 编写
- 在 Drake 中构建包含自定义物体的完整机器人仿真

你已掌握 Drake 物理仿真和自定义机器人环境的基础技能。这些能力对后续进阶内容和课程项目都至关重要！

**下一步：** 在笔记本3中，你将学习如何用 `scenario.yaml` 文件和 `HardwareStation` 快速搭建复杂场景。